In [ ]:
import sisl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
%matplotlib inline

# TI-01 Spin Texture 
In this exercise, we learn how to calculate and plot spin-textures using ```sisl``` and ```SIESTA```, at the example of bismuthene.

## Exercise Overview
1. Create bismuthene geometry.
2. Generate ```SIESTA``` Hamiltonian.
3. Calculate the band structure.
4. Calculate the spin texture.

## Exercise

#### 1. Create the geometry in ``sisl`` and save it.
We will consider bismuthene in a buckled hexagonal phase. This crystal structure is similar to that of graphene. However, the two sublattices form two parallel planes, separated by the buckling height $h$.
We will use an in-plane lattice constant $a$ of $4.60\mathring{\textrm{A}}$ and buckling height $h$ of $1.62\mathring{\textrm{A}}$. The lattice constant for the orthogonal direction $c$ can be large, e.g. $40\mathring{\textrm{A}}$.

|Top view |Side View |
|:--------|:---------|
|<img src="img/Bi2D-BHex-c.png" alt="BHex-c" style="width: 400px;"/>|<img src="img/Bi2D-BHex-a.png" alt="BHex-a" style="width: 400px;"/>|

In [ ]:
# Define lattice vectors and atomic positions
# geom = sisl.Geometry(atomic_positions, lattice_vectors, atomic_species)
geom.write('STRUC.fdf')

#### 2. Generate SIESTA Hamiltonian
A sample input file for ```SIESTA``` can be found in the *siesta_work* folder. If there are any unfamiliar flags in the input file, the manual (/Docs/siesta.pdf) can offer explanations. For the next steps the ```SIESTA``` Hamiltonian is required. So, we have to make sure that ```SIESTA``` will store it (SaveHS True or TS.HS.Save True). 
```
    siesta Bi2D_BHex.fdf > Bi2D_BHex.out
```
Check the output file to ensure that the calculation converged and no errors occured.

#### 3 Bandstructure

Use ```sisl``` to calculate the band structure along the $M$-$\Gamma$-$K$-$M$ path, using the ```SIESTA``` Hamiltonian (Bi2D_BHex.HSX). If necessary, [seeK-path](https://www.materialscloud.org/work/tools/seekpath) can help finding the right k-point path in the Brillouin Zone.

*Note: seeK-path uses a standartized unit cell. The displayed k-points refer to the reciprocal cell corresponding to this standartized cell not the original one.*

In [ ]:
# Read the Hamiltonian from siesta output and create sisl.BandStructure
# sile = sisl.get_sile('path/to/*.fdf')
# H = ...
# EFermi = ...
# kpath = sisl.BandStructure(...)

In [ ]:
# Calculate the bands
# bands = ...

In [ ]:
# Plot the band structure
# Feel free to re-use code from previous exercises

#### 4 Spin Texture

Calculate the spin texture, i.e. the spin moment of the eigenstates, along the same k-path as in 3.

1. The spin moment for a given k-point can be calculated with sisl, using the ```spin_moments``` routine See [```sisl.physics.Hamiltonian.spin_moment```](http://zerothi.github.io/sisl/docs/latest/api-generated/sisl.physics.Hamiltonian.html?highlight=spin_moment#sisl.physics.Hamiltonian.spin_moment).
```
>>> H = Hamiltonian(...)
>>> Sx, Sy, Sz = H.spin_moment().T
```
2. The ``sisl.BrillouinZone`` or ``sisl.BandStructure`` object can be used evaulate functions for all k-points. Using the ```call``` routine:
```
>>> H = Hamiltonian(...)
>>> bz = BrillouinZone(H)
>>> bz.eigh() == bz.call(H.eigh)
```

In [ ]:
# Calculate the spin moments for all eigenstates along the k-path.
# spin_moments = 

A convient way to visualize the spin texture is coloring the bands according to the spin moment. The template below can be used to plot the spin texture. 

_For reference on how to create multi coloured lines refer to the matplotlib [documentation](https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/multicolored_line.html)._

In [ ]:
def plot_spin_texture(kpath, bands, spin_moments):    
    lk = kpath.lineark()
    xtick, xtick_label = kpath.lineartick()    
    nk, nbands = bands.shape
    
    # Create a figure with three subplot one for each component of the spin moment
    fig, axes = plt.subplots(1, 3, figsize=(16, 9), dpi=300, sharex=True, sharey=True)

    # Set the range of z-values, which will determine the color.
    norm = plt.Normalize(-1, 1)

    # Iterate of the spin components
    for icomp, component in enumerate(['$S_x$', '$S_y$', '$S_z$']):
        # Iterate over all bands 
        for ibnd in range(nbands):
            # It is not possible to change the color of a line directly, so we create small 
            # line segements from one point on the x-axis to the next. These segments can 
            # then be colored individually.
            points = np.array([lk, bands[:, ibnd]]).T.reshape(-1, 1, 2)
            segments = np.concatenate([points[:-1], points[1:]], axis=1)
            # Create a collection of the segments and specify a map that assigns colors 
            # to the segments according to the z-value
            lc = LineCollection(segments, cmap='coolwarm', norm=norm)
            # Set the z-values 
            lc.set_array(spin_moments[icomp])             
            lc.set_linewidth(3)

            # Add the LineCollection to the subplot
            line = axes[icomp].add_collection(lc)
            
        axes[icomp].set_title(component)

    # All subplots share the same axis settings, so we can just adjust them once     
    ymin, ymax = (-2, 2)
    axes[0].set_xlim(min(lk), max(lk))
    axes[0].set_ylim(ymin, ymax) 
    axes[0].set_ylabel('Eigenspectrum [eV]')
    axes[0].xaxis.set_ticks(xtick)
    axes[0].set_xticklabels(xtick_label)

    for axis in axes:
        for tick in xtick:
            axis.plot([tick, tick], [ymin, ymax], 'k', linewidth=0.5)
    
    # Add a colorbar to the plot
    fig.colorbar(line, ax=axes.ravel().tolist())
    plt.show()

In [ ]:
# Use the function above to plot the spin-texture
# plot_spin_texture(...)

What happens when every other band is plotted, instead of all of them at the same time? Why?